## Project: **Finding Lane Lines on the Road** 

The goal of this project is to identify lane lines on the road from a set of images and videos.

The following image processing techniques are applied (OpenCV implementations):
- Gaussian Blurring
- Canny edge detection
- Region masking
- Hough transform

Starter code can be found at [https://github.com/udacity/CarND-LaneLines-P1](https://github.com/udacity/CarND-LaneLines-P1).

### Import Packages

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2

import os

# inline plots
%matplotlib inline 

### Read in a Single Image

In [ ]:
%pylab inline
pylab.rcParams['figure.figsize'] = (10, 12.5)

#reading in an image
image = mpimg.imread('test_images/solidWhiteRight.jpg')

#printing out some stats and plotting
print('This image is:', type(image), 'with dimensions:', image.shape)
plt.imshow(image)  # if you wanted to show a single color channel image called 'gray', for example, call as plt.imshow(gray, cmap='gray')

### Helper Functions (provided)

In [ ]:
import math

def grayscale(img):
    """Applies the Grayscale transform"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines, color=[255, 0, 0], thickness=2):
    """
    Draws `lines` with `color` and `thickness` onto 'img'.    
    Lines are drawn in-place (mutates the image).
    """
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)
            
def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

### Lane-finding Pipeline on Test Images

In [ ]:
# Specify directory names.
in_dir = 'test_images/'
subplots_dir = 'test_images_intermediate/'  # dir for intermediate images
out_dir = 'test_images_out_detect/'

# Create output directories if needed.
if not os.path.exists(out_dir):
    os.makedirs(out_dir)
if not os.path.exists(subplots_dir):
    os.makedirs(subplots_dir)

# Process each file in input directory.
for img_filename in os.listdir(in_dir):

    # Set up a subplot for keeping intermediate images.
    fig = plt.figure()
    fig.suptitle('Lane Detection on ' + img_filename)
    
    # Read in image.
    img = mpimg.imread(in_dir + img_filename)
    a = fig.add_subplot(4,2,1)
    plt.imshow(img)
    a.set_title('Original')
    
    # Convert image to grayscale.
    gray_img = grayscale(img)
    a = fig.add_subplot(4,2,2)
    plt.imshow(gray_img, cmap='gray')
    a.set_title('Grayscale')
    
    # Apply Gaussian blurring.
    for i in range(0,3):
        kernel_size = 5
        gs_gray_img = gaussian_blur(gray_img, 5)
    a = fig.add_subplot(4,2,3)
    plt.imshow(gs_gray_img, cmap='gray')
    a.set_title('Gaussian blurred')

    # Perform Canny edge detection.
    lower_thresh = 50
    upper_thresh = 150
    canny_img = canny(gs_gray_img, lower_thresh, upper_thresh)
    a = fig.add_subplot(4,2,4)
    plt.imshow(canny_img, cmap='gray')
    a.set_title('Canny edge')

    # Apply ROI masking to edge image.
    h_offset = 20
    v_offset = 50
    l_base = 0
    r_base = 960
    left_bottom = [l_base, img.shape[0]]
    left_top = [img.shape[1]/2 - h_offset, img.shape[0]/2 + v_offset]
    right_top =  [img.shape[1]/2 + h_offset, img.shape[0]/2 + v_offset]
    right_bottom = [r_base, img.shape[0]]
    
    # https://stackoverflow.com/questions/17241830/opencv-polylines-function-in-python-throws-exception/18817152#18817152)
    # ^ regarding input to cv2.fillpoly
    roi = [np.array([left_bottom, left_top, right_top, right_bottom], np.int32)]
    roi_canny_img = region_of_interest(canny_img, roi)
    a = fig.add_subplot(4,2,5)
    plt.imshow(roi_canny_img, cmap='gray')
    a.set_title('Edges after ROI mask')

    # Obtain Hough transform lines within ROI.
    rho = 1
    theta = np.pi/180
    threshold = 5
    min_line_len = 10
    max_line_gap = 5
    roi_hough_img = hough_lines(roi_canny_img, rho, theta, threshold, min_line_len, max_line_gap)
    a = fig.add_subplot(4,2,6)
    plt.imshow(roi_hough_img)
    a.set_title('Hough lines')
    
    # Overlay detected lines on top of original image.
    ann_img = weighted_img(roi_hough_img, img, 1, 2, 0)
    a = fig.add_subplot(4,2,7)
    plt.imshow(ann_img)
    a.set_title('Output')
    
    # Save intermediate and output images.
    plt.savefig(subplots_dir + 'intermediate_' + img_filename)
    mpimg.imsave(out_dir + 'annotated_' + img_filename, ann_img)


### Improved 'draw_lines' function - extrapolates lane lines

In [ ]:
def extrapolate_lines(img, lines, roi, m_thresh, color=[255, 0, 0], thickness=2):
    '''
    This is the improved 'draw_lines' function that extrapolates the lane, 
    based on provided hough_lines, slope threshold, and ROI.
    '''
    left_lane_points = []
    right_lane_points = []
    
    for line in lines:
        # Get line endpoints and calculate slope.
        x0, y0, x1, y1 = line[0]
        m = -(y1-y0) / (x1-x0)
        
        # Reject lines with slope not representative of a lane line.
        if abs(m) < m_thresh:
            continue
            
        # Classify line endpoints as part of left lane or right lane.
        if m > 0:
            left_lane_points.append([x0, y0])
            left_lane_points.append([x1, y1])
        else:
            right_lane_points.append([x0, y0])
            right_lane_points.append([x1, y1])

    # Use np.polyfit to find the line of best fit for the LEFT lane.
    if (len(left_lane_points)) > 0:
        left_lane_points = np.array(left_lane_points)
        m, b = np.polyfit(left_lane_points[:,0], left_lane_points[:,1], 1)
        bot_y = int(roi[0][0,1])
        bot_x = int((bot_y - b) / m)
        top_y = int(roi[0][1,1])
        top_x = int((top_y - b) / m)
        cv2.line(img, (bot_x,bot_y), (top_x,top_y), color, thickness)
                 
    # Use np.polyfit to find the line of best fit for the RIGHT lane.
    if len(right_lane_points) > 0:
        right_lane_points = np.array(right_lane_points)
        m, b = np.polyfit(right_lane_points[:,0], right_lane_points[:,1], 1)
        bot_y = int(roi[0][3,1])
        bot_x = int((bot_y - b) / m)
        top_y = int(roi[0][2,1])
        top_x = int((top_y - b) / m)
        cv2.line(img, (bot_x,bot_y), (top_x,top_y), color, thickness)

    return [left_lane_points, right_lane_points]
    


### Lane Extrapolation on Test Images

In [ ]:
def process_image(img):
    '''
    Outputs a color image (3 channel) for processing video below
    '''
    
    # Convert input image to grayscale.
    gray_img = grayscale(img)
    
    # Apply multiple Gaussian blurs .
    gs_gray_img = gray_img
    for i in range(0,1):
        kernel_size = 5
        gs_gray_img = gaussian_blur(gray_img, 5)
    
    # Perform Canny edge detection.
    lower_thresh = 50
    upper_thresh = 150
    canny_img = canny(gs_gray_img, lower_thresh, upper_thresh)
      
    # Apply ROI masking to edge image.
    h_offset = 20
    v_offset = 50
    l_base = 0
    r_base = 960
    left_bottom = [l_base, img.shape[0]]
    left_top = [img.shape[1]/2 - h_offset, img.shape[0]/2 + v_offset]
    right_top =  [img.shape[1]/2 + h_offset, img.shape[0]/2 + v_offset]
    right_bottom = [r_base, img.shape[0]]
    
    roi = [np.array([left_bottom, left_top, right_top, right_bottom], np.int32)]
    roi_canny_img = region_of_interest(canny_img, roi)
    
    # Obtain Hough transform lines within ROI.
    rho = 1
    theta = np.pi/180
    threshold = 20
    min_line_len = 20
    max_line_gap = 10
    roi_hough_img = hough_lines(roi_canny_img, rho, theta, threshold, min_line_len, max_line_gap)
    h_lines = cv2.HoughLinesP(roi_canny_img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)

    m_thresh = 0.4
    line_thickness = 10
    roi_lanes_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)  # empty image
    roi_lanes = extrapolate_lines(roi_lanes_img, h_lines, roi, m_thresh, (255,0,0), line_thickness)
    #roi_lanes = extrapolate_lines(roi_lanes_img, h_lines, roi, m_thresh, (0,255,0), line_thickness)
    
    # Overlay detected lines on top of original image.
    ann_img = weighted_img(roi_lanes_img, img, 1, 2, 0)    
    return ann_img
    #return roi_hough_img
    
    #ann_img = weighted_img(roi_lanes_img, roi_hough_img, 1, 2, 0)    
    #return ann_img


In [ ]:
# Specify directory names.
in_dir = 'test_images/'
out_dir = 'test_images_out_extrapolate/'

# Create output directories if needed.
if not os.path.exists(out_dir):
    os.makedirs(out_dir)
if not os.path.exists(subplots_dir):
    os.makedirs(subplots_dir)

# Process each file in input directory.
for img_filename in os.listdir(in_dir):

    # Read in image.
    img = mpimg.imread(in_dir + img_filename)

    # Delegate to pipeline.
    ann_img = process_image(img)
    
    # Save intermediate and output images.
    mpimg.imsave(out_dir + 'annotated_' + img_filename, ann_img)
    plt.imshow(ann_img)

### Lane Detection Pipeline on Videos

#### Video 1: 'solidWhiteRight.mp4'

In [ ]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

input_dir = 'test_videos/'
output_dir = 'test_videos_output/'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [ ]:
filename = 'solidWhiteRight.mp4'

#clip1 = VideoFileClip(input_dir + filename).subclip(0,5)
clip1 = VideoFileClip(input_dir + filename)
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(output_dir + filename, audio=False)

HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(output_dir + filename))

#### Video 2: 'solidYellowLeft.mp4'

In [ ]:
filename = 'solidYellowLeft.mp4'

#clip2 = VideoFileClip(input_dir + filename).subclip(0,5)
clip2 = VideoFileClip(input_dir + filename)
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(output_dir + filename, audio=False)

HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(output_dir + filename))

## [Todo] Optional Challenge

Try your lane finding pipeline on the video below.  Does it still work?  Can you figure out a way to make it more robust?  If you're up for the challenge, modify your pipeline so it works with this video and submit it along with the rest of your project!

In [ ]:
challenge_output = 'test_videos_output/challenge.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip3 = VideoFileClip('test_videos/challenge.mp4').subclip(0,5)
clip3 = VideoFileClip('test_videos/challenge.mp4')
challenge_clip = clip3.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))